## Técnicas de Agregación


Los árboles de predicción, al igual que todo modelo estadístico, sufren el
problema del equilibrio bias-varianza. Recordemos que el término bias hace
referencia a cuánto  se alejan las predicciones de un modelo respecto a los
valores reales. La varianza hace referencia a cuánto varía el
modelo  dependiendo del conjunto con el que realizamos el entrenamiento.
Cuanto mayor sea la complejidad de un modelo, se dispone de una mayor
capacidad de adaptarlo al problema que se quiere resolver, reduciendo así el
bias y mejorando su capacidad de predicción. Sin embargo, el peligro es
el de tener un modelo tan ajustado al conjunto de entrenamiento que presenta
una alta varianza y es incapaz de adaptarse a nuevas observaciones,
entonces aparece el  problema del sobreajuste. El mejor modelo es aquel que
consigue un buen equilibrio entre bias y varianza.

Estos términos se pueden trasladar a los árboles de decisión vistos en la
sección anterior. De manera general, los árboles pequeños serán capaces de
representar de manera correcta la relación existente entre las variables, por
lo que tienden a tener un alto bias, pero poca varianza. En cambio, los árboles
grandes se ajustan mucho a los datos de entrenamiento, por lo
que tienen menor bias pero una alta varianza.

Tal como se describe en la introducción del capítulo _Bagging_, _random
forests_ y _boosting_ utilizan los árboles de decisión como las piezas
elementales para construir mejores modelos de predicción consiguiendo un
mejor equilibrio entre bias y varianza.


### Bagging
**NOTA:** Traduir bagging? Wikipedia ho tradueix com Agregación de bootstrap o
empaquetado


Los árboles de decisión sufren de una gran varianza, esto significa que si
dividimos los datos de entrenamiento en dos partes de manera aleatoria, y
construimos un árbol de decisión para cada una de las dos mitades, los
árboles resultantes que obtenemos podrían ser muy diferentes. Un
procedimiento  con baja varianza producirá resultados similares si se aplica
a distintos conjuntos de datos.

La técnica que aquí se presenta, __bagging__, es un procedimiento diseñado para
reducir la varianza de un método de aprendizaje estadístico. Dado un
conjunto de $n$ observaciones independientes $Z_1, \ldots, Z_n$ cada una de
ellas con su propia varianza $ \sigma^2$, la varianza de la media de estas
observaciones $\overline{Z}$ es  $ \frac{\sigma^2}{n}$. Por lo
tanto, una forma natural de reducir la varianza y, por lo tanto, aumentar la
precisión de la predicción de un método de aprendizaje es construir
diferentes conjuntos de entrenamiento, construir un modelo de
predicción separado usando cada uno de los conjuntos y realizar el promedio
de las predicciones resultantes.

Desgraciadamente, normalmente no tenemos suficientes datos para crear
diferentes conjuntos de entrenamiento. Es por este motivo que se aplica la
técnica de __bootstrap__, simularemos los diferentes conjuntos de
entrenamiento necesarios para construir diferentes árboles y reducir la
varianza del clasificador. Esto se consigue extrayendo repetidos subconjuntos
de la muestra original. Estos subconjuntos muestrales deben extraerse
utilizando  un muestreo con reposición, de tal forma que algunos elementos no
serán  seleccionados y otros lo podrán ser más de una vez en cada muestreo.
(_refereciar al Hasting capítol 5_)

**TODO**: gràfic mostrant bootstrap

Para aplicar la técnica de _bagging_ construiremos un conjunto de árboles de
regresión usando diferentes conjuntos de entrenamiento con la técnica
explicada anteriormente y daremos como resultado final el promedio de las
predicciones individuales de cada uno de los árboles que forman el conjunto.
(_Estos árboles crecen profundamente y  no se podan_) . Al promediar el
resultado de cada uno de estos árboles evidentemente, se consigue reducir la
varianza. Se ha demostrado que el _bagging_ proporciona grandes mejoras  en la
precisión de los resultado al combinar un gran número de árboles en un
solo procedimiento.

Hasta ahora, hemos descrito el procedimiento de _bagging_ en  el caso de
necesitar solventar un problema aplicando una regresión. Esta técnica también
es aplicable a problemas de clasificación, en este caso se
toma la clase que más clasificadores (árboles) hayan predicho.

**NOTA:** Integrar això, potser s'hauria de'eXplicar en la secció anterior.
Bias-variance tradeoff

_Según lo explicado anteriormente, sabemos que cada árbol
individual tiene una alta varianza, pero un bajo sesgo (xq son profundos). Al
obtener información agregada del conjunto  árboles se reduce la varianza._


#### Cálculo de el error (_out-of-bag_)


Cuando usamos la técnica de _bootstrapping_ existe una manera  sencilla de
estimar el error del método sin necesidad de realizar validación cruzada
o crear un subconjuto de validación.

El hecho de  que los árboles se ajusten de forma repetida empleando muestras
generadas por _bootstrapping_ conlleva que, en promedio, cada árbol solamente
usa alrededor de dos tercios de las observaciones originales. Al
tercio restante se le llama _out-of-bag_ (OOB). Si para cada árbol construido
se registran las observaciones empleadas, se puede predecir la respuesta de
una observación haciendo uso de aquellos árboles en los que esa observación
ha sido excluida (OOB) y promediándolos si queremos realizar una tarea de
regresión o obteniendo la moda en el caso que queramos realizar una tarea de
clasificación.

Siguiendo este proceso, se pueden  obtener las predicciones para las todas
las observaciones y con ellas calcular el conocido como _OOB-mean square error_
para casos de regresión o el _OOB-classification error_ para árboles de
clasificación. Como la variable respuesta de cada observación se predice
empleando únicamente los árboles en cuyo ajuste no participó dicha
observación,  este cálculo sirve como estimación del
error del conjunto de test.  (**Depen de si s'ha explicat o no**) _De hecho, si
el número de árboles es suficientemente alto, el OOB-error es prácticamente
equivalente al leave-one-out cross-validation error_.  Esto evita
tener  que recurrir al proceso de cross-validation para la optimización de
los hiperparámetros.


#### Un ejemplo de _bagging_

Como hemos explicado anteriormente esta técnica es aplicable a diferentes
métodoes de aprendizaje. En el paquete _sklearn.ensemble_ de Scikit
encontramos las clases _BaggingClassifier_ y _BaggingRegressor_ que usan los
árboles de decisión como técnica por defecto.

Vamos a ver un ejemplo de regresión con esta técnica:

In [80]:
from sklearn.ensemble import BaggingRegressor
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
import numpy as np

X, y = make_regression(n_samples=50, n_features=1,
                       n_informative=1, n_targets=1, noise=10,
                       random_state=0, shuffle=True)

elem = np.linspace(np.min(X),np.max(X),150).reshape(-1, 1)


regressor = DecisionTreeRegressor(random_state=33)
regressor.fit(X, y)

predicciones = regressor.predict(elem)

regr = BaggingRegressor(n_estimators=100, random_state=0)
regr.fit(X, y)

y_predict= regr.predict(elem)


In [ ]:
#plt.scatter(X, y);
#plt.plot(elem, y_predict,label="bagging");
#plt.plot(elem, predicciones, label="regressor tree")
#plt.legend()


### Random Forest

Los _random forests_ son una técnica que al igual que la técnica de
_bagging_ consiste en la agregación de diversos árboles de decisión
para proporcionar una decisión conjunta creando diferentes conjuntos de
entrenamiento  mediante _bootstrapping_. Esta nueva técnica que presentamos
consigue una mejora de rendimiento respecto a la técnica  de _bagging_.

La particularidad que diferencia a los _random forest_ es que al construir
cada  uno de los árboles de decisión, cada vez que se realiza una división de
un  nodo en un árbol, se elige de manera  aleatoria un subconjunto, $m$, de
características. Este nueva división solo podrá ser realizada usando una
característica que pertenezca a $m$. Cada vez que se realiza una nueva
división se vuelve a generar este conjunto, el valor por defecto de $m \simeq
\sqrt{p}$ donde $p$ es el número total de características.


En resumen, al construir un _random forest_, el algoritmo no considerará la
mayoría de las características disponibles en cada nueva división de un nodo
del árbol. Esto permite construir árboles que serán muy diferentes entre si y
 también evitar que existan características que dominantes.

Una de las problemáticas que surgen del uso de la técnica de _bagging_ es la
similitud de todos los árboles construidos. Desafortunadamente, realizar
promedio de muchos valores altamente correlacionados no conduce a una gran
reducción de la varianza como promediar muchos valores no correlacionados.
Esto significa que la técnica de _bagging_ no proporcionará una gran
reducción de la varianza comparando con la que tenemos con un solo árbol.
Los _random forest_ estan diseñados para  solventar esta problemática al
forzar  cada división a considerar un subconjunto de todas las
características disponibles.

La principal diferencia entre _bagging_ y _random forest_ es  la elección del
tamaño del subconjunto del predictor, si un  _random forest_ se
construye usando  $m = p$, esto equivale a realizar la técnica de
_bagging_.


### Boosting

La última técnica que vamos a describir en este capítulo es el _boosting_,
que también servirá para mejorar los resultados de predicción de los
árboles de decisión. Al igual que en la técnica de  _bagging_, el
_boosting_ es un enfoque general que se puede aplicar a diversos métodos de
aprendizaje automático para tareas de regresión o de clasificación.

Cuando aplicamos _bagging_ creamos múltiples subconjuntos usando
_bootsrapping_ y entrenamos un árbol para cada uno de estos subconjuntos,
luego combinamos todos los árboles para crear
un modelo predictivo único. La técnica de _boosting_  funciona de una manera
similar, excepto que en este caso los árboles crecen de manera  secuencial.
La construcción de cada árbol se basa en información de los árboles construidos
previamente. En _boosting_ no construimos diferentes conjuntos de
entrenamiento usando _bootstrapping_ sino que cada árbol se ajusta sobre una
versión modificada del conjunto de datos original.

Se dice que la técnica de _boosting_ implica un  aprendizaje lento. Dado el
estado del modelo actual a partir  de sus residuos. Para un número de árboles
al que llamaremos $B$, agregamos predicciones de árboles en de manera
iterativa. Es decir, agregamos un segundo árbol para mejorar el rendimiento
del primer árbol o para intentar corregir el error del primer árbol, y así
sucesivamente. El proceso a realizar consiste en que hacemos es restar la
predicción del primer modelo multiplicada por una constante $0 <λ \leq 1$ de
los valores objetivo y luego, tomando estos valores como valor objetivo,
ajustamos el segundo modelo, y así sucesivamente. Podemos verlo como: nuevos
 modelos que intentan corregir modelos anteriores / errores del modelo anterior.

En este caso, los árboles que forman el modelo suelen ser bastante
pequeños, poco profundos, comparando con las técnicas anteriores. Cuentan con
con pocos  nodos terminales. Estos estan determinados por el parámetro $d$
del  algoritmo.

Ajustando árboles pequeños a los residuos de los árboles anteriormente
entrenados, mejoramos lentamente en áreas donde hasta ahora  no funcionaba
de manera precisa. El parámetro  $λ$ ralentiza el proceso, lo que permite
crear más árboles que ataquen a los  residuos.

El aumento de los árboles de clasificación se realiza de una manera similar
pero un poco más compleja, y sus detalles no entran en el ámbito de este material.

Existen tres parámetros que permiten ajustar esta técnica:

1. El número de árboles $B$. A diferencia de las técnicas de _bagging_ y  de
_random forest_, esta técnica puede llegar al sobreajuste (__overfitting__)
sobre el conjunto de entrenamiento si $B$ es demasiado grande, aunque este
tiende a ocurrir lentamente. Para seleccionar el mejor parámetro de $B$ se
recomienda usar la validación cruzada.

2. El parámetro de contracción $λ$, es un positivo de valor muy pequeño.
Este controla la velocidad a la que aprende el método. Los valores típicos son 0
.01 o 0.001, su elección final depende del problema. Si
elegimos un valor muy pequeño para $λ$ normalmente necesitaremos
seleccionar un valor muy grande de $B$ para lograr un buen rendimiento.

3. El número de divisiones en cada árbol, $d$, lo que controla la
complejidad del conjunto potenciado. Normalmente, usar $d = 1$ funciona bien,
en cuyo caso cada árbol es un volquete, que consta de una sola división. En
este  caso, el conjunto de columnas aumentadas se ajusta a un modelo aditivo,
ya que cada término implica solo una variable única. De manera más general,
la profundidad de interacción y los controla el orden de interacción del
modelo impulsado, ya que $d$ divisiones pueden involucrar a la mayoría de $d$
variables.



### Determinar la importancia de las características






## Un ejemplo completo

**NOTA:** aquí la meva idea seria fer un exemple de classificació comparant
arbres simples amb alguna de les tècniques anteriors. Fer selecció dels
millors parametres i mostrar com canvia el resultat amb el nombre d'estimadors.

## Resumen


* Comentar que tant en els arbres individuals com en les tècnicques
d'agregació ne le cas de classif es pot donar %

* En resumen, el  promedio de un conjunto de observaciones permite reducir la
varianza.




## Ejercicios